#Lab.02 / IBM3202 – Visualizando y Comparando Estructuras Moleculares en Google Colab usando py3Dmol

##Aspectos teóricos


Como se describio en el **Lab.01**, el archivo PDB contiene la información de todas las condiciones experimentales que permitieron obtener una estructura proteica tridimensional (contenida en el archivo PDB), como también información acerca de las características biológicas de la proteína y las características generales del cristal (celda unitaria, dimensiones, número de monómeros, ensamble biológico, etc.). Sin embargo, su contenido más relevante corresponde a las **coordedanas** (las posiciones tridimensionales) de los átomos de la estructura de una proteína dada. Esto significa, que todos los archivos PDB contienen el siguiente formato mínimo:

![alt text](https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pdbformat_01.png)

Del cual **ATOM** (o HETATM) indica que la línea contiene las coordenadas atómicas, un número único para cada átomo (a esta lista se le llama index), el tipo de átomo, el residuo al cuál pertenece el átomo, la cadena polipeptídica a la cual pertenece la molécula, la posición o número de residuos en la estructura primaria, y las coordenadas cartesianas para cada átomo. Cualquier molécula (proteína o no) se puede escribir en este formato siempre y cuando tengamos las coordenadas cartesianas para cada átomo.

Como recordarás de las clases, las estructuras biológicas son obtenidas por diferentes evidencias experimentales: **cristalografía de rayos X**, **Resonancia Magnética Nuclear** o, más recientemente, **Crio microscopía electrónica**. Esta información está lejos de entregar resoluciones atómicas exactas, por lo tanto mediciones de qué tan bien definido son las coordenadas también se anotan en este formato. Aquellas columnas, posteriores a las coordenadas, son la ocupancia y factor de temperatura (o B). La ocupancia nos dice si el átomo está definitivamente en esa posición, o si alguna conformación alternativa es probable. La imagen de la **Figura 1** presenta este escenario, en el cual una proteína resuelta por cristalografía de rayos X (PDB 6ANE; la misma que usamos en el Lab.01), donde un residuo de triptófano particular no puede ser asignado a una única configuración (es decir, la densidad electrónica se ajusta igual de bien a ambas configuraciones). En este caso, esto se interpreta como que el residuo realmente _ocupa_ ambas configuraciones en el cristal, de tal modo que cerca de la mitad de las moléculas en la red cristalina toma uno de estos confórmeros.

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pdbformat_01.gif'/>
<figcaption>FIGURA 1. Ocupancias de un residuo triptofano en la estructura cristalográfica (PDB ID 6ANE). En este caso, el mismo átomo tiene dos conjuntos de coordenadas, una por cada configuración. Por lo tanto, la ocupancia de cada una de estas líneas de coordenadas se reduce a 0.50. En general, cuando un mismo átomo (definido en su línea de coordenadas) tiene _n_ posibles configuraciones, su ocupancia se reduce a 1/n.</figcaption></center>
</figure>

💡 **Pista:** En el GIF de la Figura 1 se muestra el resultado de comparar el mismo residuo de triptófano después superposición estructura de dos cadenas diferentes obtenidas en el mismo experimento de cristalografía de rayos X. Esto iluustra la necesidad de inspeccionar cuidadosamente la estructura PDB previo a cualquier estudio que se realice con ella, debido a que si hay diferencias en una cadena lateral que está cerca del sitio activo esto puede afectar los resultados de estudios posteriores como docking molecular.

Por otro lado, los **factores de _temperatura_** son una medida de la incertidumbre en las posiciones asignadas a cada átomo. En el caso previo, la ocupancia nos dice que teníamos múltiples configuraciones estables dentro de la estructura. En vez, los factores de temperatura (también llamados valores B isotrópicos) están midiendo qué tan probable is observar aquél átomo en la posición dada. Mientras mayor es este valor, mayor es la mobilidad acerca de la posición definida, por lo tanto mayor la incertidumbre. Este valor depende inversamente de la resolución: estructuras de alta resolución tienen posiciones atómicas claramente definidas, y podemos ver con escasa incertidumbre dónde cada átomo debiera ser ubicado. Por otro lado, estructuras de baja resolución sólo nos muestran las características gruesas del ensamble molecular, y las coordenadas son mucho más difíciles de asignar **(Figura 2)**.

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pdbformat_02.png'/>
<figcaption>FIGURA 2. Efecto de la resolución en los mapas de densidad electrónica. En el primer caso podemos er claramente las posiciones de los átomos pesados, mientras que a menores resoluciones estas posiciones se vuelven inciertas.</figcaption></center>
</figure>

Los dos parámetros recién definidos son complementarios entre sí, ya que múltiples confórmeros estables que emergen de ocupancia incompleta no son necesariamente igualmente probables, por lo que la información mutua ayuda a decidir cuál es el más probable **(Figura 3)**.

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pdbformat_03.png'/>
<figcaption>FIGURE 3. Coordinate lines for the same atom (CB) in two distinct tryptophan configurations (ATRP, BTRP). As you may notice, the occupancy is half for each, yet the B-factor is lower for the first (A) configuration, hence more likely.</figcaption></center>
</figure>


With this, we have briefly covered the essentials of PDB structures. However, the goodness of a given protein structure deposited in the PDB is present in the webpage of the corresponding PDB identifier, as show in **Figure 4**.

<figure>
  <center>
    <img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pdbformat_04.png'/>
    <figcaption>FIGURA 4. Calidad general de la estructura. Este gráfico depende de la estructura y la resolución. Clasifica las estructuras desde las peores (rojo) hasta las mejores (azul) a la misma resolución.</figcaption>
  </center>
</figure>


<p align="justify">Varios parámetros permiten evaluar la calidad de una estructura de proteína resuelta por difracción de rayos X.

>En **Rwork/Rfree**, el primero indica qué tan bien se ajustan las coordenadas atómicas a los datos experimentales, mientras que el segundo es una medida de "control" con parte de los datos que se excluyeron al calcular Rwork. Cuanto más bajos sean estos valores, mejor será el ajuste de la estructura resuelta a la densidad electrónica. Sin embargo, tenga cuidado si Rwork << Rfree, ya que es probable que los datos se estén ajustando demasiado a la densidad electrónica experimental.

>El **Clashscore** mide si los átomos están inusualmente cerca entre sí (número de choques por cada 1,000 átomos).

>Los **Ramachandran outliers** miden si hay torsiones inusualmente estiradas en los ángulos dihedrales φ y ψ del esqueleto.

>Las cadenas laterales de aminoácidos también adoptan configuraciones preferidas (o rotámeros), por lo que los **Sidechain outliers** miden cuántos rotámeros no comunes están presentes en una estructura dada.

>Finalmente, **RSRZ** es una medida entre el modelo atómico y los datos de espacio real.

Aunque el archivo PDB contiene las coordenadas de una estructura dada en texto, ya has visto que la mayoría de las estructuras de proteínas resueltas se muestran típicamente como representaciones visuales de esas coordenadas atómicas.

Aquí, utilizaremos **py3DMol** (paquete de Python para visualización molecular sin dependencias en cuadernos iPython) para visualizar y analizar algunas de estas estructuras de proteínas. Cabe destacar que este paquete será nuestra herramienta fundamental de análisis para la mayoría de los tutoriales futuros relacionados con el trabajo con estructuras de proteínas.</p>


##Aspectos experimentales

###Parte I. Obteniendo y visualizando estructuras del PDB

<p align="justify">Puedes obtener estructuras PDB desde su sitio web (www.rcsb.org). Alternativamente, puedes utilizar directamente la terminal para descargar un archivo PDB dado con el código de acceso conocido, como se muestra a continuación, donde XXXX debe ser reemplazado por el código PDB de 4 letras:

```
!wget http://www.rcsb.org/pdb/files/XXXX.pdb.gz
!gunzip XXXX.pdb.gz
```




In [ ]:
##Escribe tu código aquí para obtener una proteína de tu elección!

# Sustituye 'XXXX' con el código PDB de 4 letras de la proteína que deseas descargar
!wget http://www.rcsb.org/pdb/files/XXXX.pdb.gz
!gunzip XXXX.pdb.gz


Si abres uno de estos archivos con un editor de texto (como hicimos en el Lab.01), podrás notar todas las cosas que discutimos en el Lab.01. Básicamente, encontrarás mucha información relacionada con el experimento, incluidos los parámetros de extracción y resolución de la estructura cristalina, seguidos de la información de las coordenadas de cada átomo que se asignó a una densidad electrónica dada después del procedimiento de difracción de rayos X y resolución de la estructura (notando que esto significa que los átomos sin densidad electrónica no estarán presentes en el archivo PDB).

In [ ]:
# ¿Quieres saber cuántas cadenas contiene el archivo PDB que acabas de descargar? ¡grep puede ayudarte!
# (recuerda verificar tu directorio)
!grep 'COMPND'.*'CHAIN' 6ANE.pdb

# ¿Cuántos residuos tiene cada cadena? ¡Ahora podemos usar awk!
!awk '$1=="ATOM" && $3=="CA" && $5=="A" {print $0}' 6ANE.pdb | wc -l


Vamos a usar [**BioPython**](https://biopython.org/) (un conjunto de herramientas de libre acceso para cómputo biológico escritas en Python) para ayudarnos a obtener información del PDB para cada cadena.

Como el archivo `6ane.pdb` tiene 3 cadenas diferentes, aquí utilizamos el módulo **BioPDB** de BioPython para dividir nuestro archivo PDB multicadena en tres archivos diferentes:



In [ ]:
# La siguiente línea es una forma común de instalar herramientas/módulos de Python
!pip install biopython

# Aquí importamos Bio.PDB para usarlo en la manipulación de archivos PDB
from Bio.PDB import *
# Y configuramos un analizador para nuestro PDB
parser = PDBParser()
io = PDBIO()
structure = parser.get_structure('X', '6ANE.pdb')

# Esto separará cada cadena en su propio archivo PDB
for chain in structure.get_chains():
    io.set_structure(chain)
    io.save("6ANE_" + chain.get_id() + ".pdb")


Para visualizar esta estructura, cargaremos nuestro archivo PDB (es decir, 6ane) en **py3Dmol**, una implementación en Python de [**3Dmol.js**](http://3dmol.csb.pitt.edu/). Puedes leer la documentación completa de 3Dmol.js [aquí](https://3dmol.csb.pitt.edu/doc/index.html), que es muy similar a la que vamos a usar en este tutorial, con algunas adiciones adicionales aquí y allá.


In [ ]:
#Aquí instalamos py3Dmol
!pip install py3Dmol

In [ ]:
# Y ya aprendimos cómo importar estos módulos, ¿verdad?
import py3Dmol

#####Visualicemos algunos PDBs!

En la siguiente celda de código, echamos un vistazo línea por línea a los diferentes comandos necesarios para visualizar dos de las cadenas que extrajiste de 6ANE. Ten en cuenta que el texto dentro de cada paréntesis sigue la estructura de un formato de datos JSON, que se revisa en detalle [aquí](https://towardsdatascience.com/an-introduction-to-json-c9acb464f43e).

Presta especial atención a las siguientes clases de **py3Dmol** y lo que hacen: _**addmodel**_, _**setBackground**_ y _**setStyle**_.

💡 **CONSEJO:** La sintaxis para cargar diferentes estructuras y propiedades es realmente sencilla y autoexplicativa. Aun así, recuerda tener cuidado al abrir y cerrar {} y () y al usar comillas cuando sea necesario.



In [ ]:
#Primero asignamos py3Dmol.view a la variable view
view = py3Dmol.view()
#Las siguientes líneas se utilizan para agregar la clase addModel
#para leer los archivos PDB de las cadenas B y C
view.addModel(open('6ANE_B.pdb', 'r').read(), 'pdb')
view.addModel(open('6ANE_C.pdb', 'r').read(), 'pdb')
#Hacemos zoom en todas las estructuras visualizadas
view.zoomTo()
#Aquí establecemos el color de fondo como blanco
view.setBackgroundColor('white')
#Aquí establecemos el estilo de visualización para las cadenas B y C
view.setStyle({'chain':'B'},{'cartoon': {'color':'purple'}})
view.setStyle({'chain':'C'},{'cartoon': {'color':'yellow'}})
#Y finalmente visualizamos las estructuras con el comando de abajo
view.show()


Como podemos ver, podemos cargar las cadenas B y C de 6ANE en representación de dibujos animados, pero están lejos entre sí. En caso de que deseemos comparar las conformaciones de diferentes residuos en estas dos cadenas (como la conformación del residuo de triptófano inestable mencionado anteriormente), vamos a utilizar nuevamente el módulo de Python Bio.PDB, implementado en el siguiente script, para:
1. Superponer la cadena B de 6ANE sobre la cadena C
2. Imprimir el RMSD
3. Guardar el PDB superpuesto de la cadena B


In [ ]:
#El siguiente código fue creado por Anders Steen Christensen
#de la Universidad de Basilea y está disponible en
#https://gist.github.com/andersx/6354971

import Bio.PDB
import os

# Selecciona los números de residuos que deseas alinear
# y colócalos en una lista
start_id = 1
end_id   = 262
atoms_to_be_aligned = range(start_id, end_id + 1)

# Inicia el analizador
pdb_parser = Bio.PDB.PDBParser(QUIET = True)

# Obtén las estructuras
ref_structure = pdb_parser.get_structure("reference", "6ANE_C.pdb")
sample_structure = pdb_parser.get_structure("sample", "6ANE_B.pdb")

# Utiliza el primer modelo en los archivos pdb para el alineamiento
# Cambia el número 0 si deseas alinear con otra estructura
ref_model    = ref_structure[0]
sample_model = sample_structure[0]

# Crea una lista de los átomos (en las estructuras) que deseas alinear.
# En este caso, utilizamos átomos de CA cuyos índices están en el rango especificado
ref_atoms = []
sample_atoms = []

# Itera sobre todas las cadenas en el modelo para encontrar todos los residuos
for ref_chain in ref_model:
  # Itera sobre todos los residuos en cada modelo para encontrar los átomos adecuados
  for ref_res in ref_chain:
    # Verifica si el número de residuo (.get_id()) está en la lista
    if ref_res.get_id()[1] in atoms_to_be_aligned:
      # Añade el átomo CA a la lista
      ref_atoms.append(ref_res['CA'])

# Haz lo mismo para la estructura de ejemplo
for sample_chain in sample_model:
  for sample_res in sample_chain:
    if sample_res.get_id()[1] in atoms_to_be_aligned:
      sample_atoms.append(sample_res['CA'])

# Ahora iniciamos el superponedor:
super_imposer = Bio.PDB.Superimposer()
super_imposer.set_atoms(ref_atoms, sample_atoms)
super_imposer.apply(sample_model.get_atoms())

# Imprime el RMSD:
print('El RMSD calculado es:')
print (str(super_imposer.rms) + ' Å')

# Guarda la versión alineada de una de las cadenas de 6ANE
io = Bio.PDB.PDBIO()
io.set_structure(sample_structure)
io.save("6ANE_B_aligned.pdb")


Ahora podemos cargar la estructura superpuesta en **py3dmol** con la celda de código que se muestra a continuación. Intenta familiarizarte con estas líneas cambiando los colores de la visualización a tu gusto.


In [ ]:
view=py3Dmol.view()
view.addModel(open('6ANE_B_aligned.pdb', 'r').read(),'pdb')
view.addModel(open('6ANE_C.pdb', 'r').read(),'pdb')
view.zoomTo()
view.setBackgroundColor('white')
view.setStyle({'chain':'B'},{'cartoon': {'color':'purple'}})
view.setStyle({'chain':'C'},{'cartoon': {'color':'yellow'}})
view.show()

Ahora queremos inspeccionar los diferentes conformeros del Triptófano 158 entre las cadenas B y C de 6ANE. Para ello, podemos seleccionar residuos específicos utilizando clases _**addStyle**_ como se muestra a continuación.

💡 **CONSEJO:** Puedes utilizar diferentes esquemas de color para representaciones de tipo palo (stick). Estos se pueden usar para controlar el color de los átomos de carbono y generalmente se indican con un sufijo "Carbon". Por ejemplo, _**blueCarbon**_ muestra todos los átomos de carbono de una representación de tipo palo en azul, _**greenCarbon**_ muestra todos los átomos de carbono en verde, etc.




In [ ]:
view=py3Dmol.view()
view.addModel(open('6ANE_B_aligned.pdb', 'r').read(),'pdb')
view.addModel(open('6ANE_C.pdb', 'r').read(),'pdb')
view.zoomTo()
view.setBackgroundColor('white')
#Establecer un estilo de visualización para la cadena B
view.setStyle({'chain':'B'},{'cartoon': {'color':'purple'}})
#Agregar un estilo de visualización para el residuo 158 en la cadena B
view.addStyle({'chain':'B','resi':158},{'stick':{'colorscheme':'grayCarbon'}})
#Establecer un estilo de visualización para la cadena C
view.setStyle({'chain':'C'},{'cartoon': {'color':'yellow'}})
#Agregar un estilo de visualización para el residuo 158 en la cadena C
view.addStyle({'chain':'C','resi':158},{'stick':{'colorscheme':'skyblueCarbon'}})
view.show()


A veces, es posible que deseemos mostrar varios residuos en nuestra visualización. En lugar de escribir una línea de código por residuo, py3Dmol también puede trabajar con selecciones de residuos:


* Una secuencia de residuos se puede dar como un rango utilizando corchetes cuadrados:


> ["158-168"]

* Lo mismo se puede hacer para residuos no secuenciales utilizando números separados por comas:


>[158,168,129,182]

Como podemos mostrar los residuos catalíticos de 6ANE, que corresponden a los números 133, 179 y 210? Y colorear al triptofano 158 en color diferente?

**Hazlo por tu cuenta** al escribir apropiadamente estos residuos y su esquema de colores en las próximas líneas de código.

In [ ]:
view=py3Dmol.view()
view.addModel(open('6ANE_B_aligned.pdb', 'r').read(),'pdb')
view.addModel(open('6ANE_C.pdb', 'r').read(),'pdb')
view.zoomTo()
view.setBackgroundColor('white')
#Asignando el color a cadena B
view.setStyle({'chain':'B'},
              {'cartoon': {'color':'purple'}}
             )
#Añade tu selección de residuos y elige un esquema de colores en la siguiente línea
view.addStyle({'chain':'B','resi':CHANGE ME},
              {'stick':{'colorscheme':CHANGE ME}})
#Estableciendo estilo para la cadena C
view.setStyle({'chain':'C'},
              {'cartoon': {'color':'yellow'}}
             )
#Agregar una representación diferente para el residuo 158 con un esquema de color diferente
view.addStyle({'chain':'C','resi':CHANGE ME},
              {'stick':{'colorscheme':CHANGE ME}}
             )
view.show()

¿Qué pasa si necesito seleccionar y mostrar residuos **dentro de un radio de distancia** de Trp158? Puedes usar selecciones **_within_** en tus clases _addStyle_, como se muestra en las siguientes líneas de código


In [ ]:
view=py3Dmol.view()
view.addModel(open('6ANE_B_aligned.pdb', 'r').read(),'pdb')
view.addModel(open('6ANE_C.pdb', 'r').read(),'pdb')
view.zoomTo()
view.setBackgroundColor('white')
#Estableciendo estilo para cadenas B y C
view.setStyle({'chain':'B'},
              {'cartoon': {'color':'purple'}})
view.setStyle({'chain':'C'},
              {'cartoon': {'color':'yellow'}})
#Mirar los residuos que están a distancia X del 158
view.addStyle({'within':{'distance': 7,
                         'sel':{'resi':158}
                         }
               }
              ,{'stick':{'colorscheme':'greenCarbon'}
                }
              )
#Despues de la selección, cambia color para los triptofanos
view.addStyle({'chain':'B','resi':158},
              {'stick':{'colorscheme':'blueCarbon'}})
view.addStyle({'chain':'C','resi':158},
              {'stick':{'colorscheme':'skyblueCarbon'}})
view.show()

¿Y sobre otras representaciones, como la representación de **van der Waals** de los átomos? Puedes mostrar la superficie del átomo utilizando vDW a través de la clase _**addSurface**_, como se muestra en la siguiente línea de código y se usa en la celda de código a continuación.

```
view.addSurface(py3Dmol.VDW,{'opacity':0.7,'color':'white'}, {'chain':'A'})
```

In [ ]:
view=py3Dmol.view()
view.addModel(open('6ANE_B_aligned.pdb', 'r').read(),'pdb')
view.addModel(open('6ANE_C.pdb', 'r').read(),'pdb')
view.zoomTo()
view.setBackgroundColor('white')
view.setStyle({'chain':'B'},{'cartoon': {'color':'blue'}})
view.setStyle({'chain':'C'},{'cartoon': {'color':'skyblue'}})
view.addStyle({'within':{'distance': 7, 'sel':{'resi':158}}},{'stick':{'colorscheme':'grayCarbon'}})
view.addStyle({'chain':'B','resi':158},{'stick':{'colorscheme':'purpleCarbon'}})
view.addStyle({'chain':'C','resi':158},{'stick':{'colorscheme':'greenCarbon'}})
#VDW Surface
view.addSurface(py3Dmol.VDW,{'opacity':0.7,'color':'white'}, {'chain':'B'})
view.addSurface(py3Dmol.VDW,{'opacity':0.7,'color':'yellow'}, {'chain':'C'})
view.show()

Finally, if you want to identify the residues in your visualization, you can also add **text labels** using the _**addLabel**_ class. However, we first need to know the names of the residues that we would like to label.

The first code cell shows how to easily obtain the name for residue number 158. If all works well, you should get "TRP" as an answer, as you already know.

The second code cell incorporates the _**addLabel**_ class to add a "W158" text label for residue 158.

In [ ]:
# Para conocer el tipo de residuo de los residuos seleccionados, ¡podrías usar awk!
# Hay formas más elegantes de hacer esto, pero por ahora...
!awk '$6=="158" && $3=="CA" {print $4}' 6ANE_B.pdb
# Inténtalo tú mismo para los residuos 210, 179 y 133

In [ ]:
# Crea una vista de py3Dmol
vista = py3Dmol.view()

# Agrega los modelos para las cadenas B y C
vista.addModel(open('6ANE_B_aligned.pdb', 'r').read(), 'pdb')
vista.addModel(open('6ANE_C.pdb', 'r').read(), 'pdb')

# Realiza un zoom para ajustar la vista
vista.zoomTo()

# Establece el color de fondo
vista.setBackgroundColor('white')

# Establece el estilo para la cadena B con un color púrpura
vista.setStyle({'chain': 'B'}, {'cartoon': {'color': 'purple'}})

# Agrega un estilo para mostrar los átomos de la cadena B en la posición 158 con un esquema de color azul
vista.addStyle({'chain': 'B', 'resi': 158}, {'stick': {'colorscheme': 'blueCarbon'}})

# Establece el estilo para la cadena C con un color amarillo
vista.setStyle({'chain': 'C'}, {'cartoon': {'color': 'yellow'}})

# Agrega un estilo para mostrar los átomos de la cadena C en la posición 158 con un esquema de color azul cielo
vista.addStyle({'chain': 'C', 'resi': 158}, {'stick': {'colorscheme': 'skyblueCarbon'}})

# Agrega una etiqueta con el texto "W158" en la posición 158
vista.addLabel("W158", {'fontOpacity': 1}, {'resi': 158})

# Puedes descomentar la siguiente línea para hacer que el modelo gire
# vista.spin("y")

# Muestra la estructura
vista.show()

# TRUCO EXPERIMENTAL
# El siguiente truco establece una entrada y un retraso de 10 segundos para imprimir una imagen PNG
# Puedes hacer clic derecho en la imagen para guardarla
# NOTA: Hay un error en Safari que genera la imagen PNG boca abajo

# importa el módulo time
# import time
# input("Presiona enter. Ahora tienes 10 segundos para elegir tu visualización")
# vista.show()
# time.sleep(10)
# vista.png()

####Un pequeño reto! (Tarea para la casa)

Intenta etiquetar y visualizar los residuos catalíticos (que han sido señalados en este tutorial) en un nuevo pedazo de código.

In [ ]:
#Inténtalo aquí

**Este es el final del segundo tutorial. ¡Buena ciencia!**

# Apéndice A. *Cargar y observar una trayectoria de MD*

En este tutorial pudimos cargar y visualizar estructuras estáticas con diferentes tipos de representación y colores. Sin embargo, en el futuro cercano estaremos realizando **Simulaciones de Dinámica Molecular**, para lo cual necesitaremos visualizar **trayectorias de simulación** (es decir, los movimientos de los átomos en nuestra simulación). En este apéndice, cubriremos algunos enfoques para lograr este objetivo.




## 1. Uso de los paquetes py3Dmol y MDAnalysis

*Esta parte del tutorial contiene una sección modificada del proyecto [Making it rain: cloud-based molecular simulations for everyone](https://dx.doi.org/10.33774/chemrxiv-2021-9f2m5). El contenido de este trabajo está disponible en https://github.com/pablo-arantes/Making-it-rain.*

Aunque **3Dmol** no fue diseñado para visualizar simulaciones de dinámica molecular, puedes crear una animación a partir de un conjunto de archivos PDB que ilustra cómo cambia la posición de los átomos de la proteína a lo largo del tiempo de simulación. Primero descargaremos un ejemplo de una simulación de MD desde el repositorio de IBM3202 en GitHub e instalaremos el paquete **MDAnalysis** desde PyPi.

📗 **NOTA:** Aprenderemos más sobre MDAnalysis en los tutoriales relacionados con la Dinámica Molecular.



In [ ]:
#!pip install MDAnalysis py3Dmol
!wget https://github.com/pb3lab/ibm3202/raw/master/files/md_files/1ihv_mon_protPBC.xtc
!wget https://github.com/pb3lab/ibm3202/raw/master/files/md_files/1ihv_mon_protPBC.gro

In [ ]:
import MDAnalysis as mda
import py3Dmol

Para visualizar una simulación de Dinámica Molecular, necesitarás un archivo con la **topología** de la proteína simulada (esto podría ser un *pdb, gro*, etc., y en este caso corresponde al archivo `1ihv_mon_protPBC.gro`) y un archivo con la **trayectoria** (`1ihv_mon_protPBC.xtc`). Crearemos variables con la ruta relacionada con cada uno:


In [ ]:
top = '1ihv_mon_protPBC.gro'
traj_end = '1ihv_mon_protPBC.xtc'

La clase `MDAnalysis.Universe` nos permite leer los resultados de la simulación. El primer argumento de la función corresponde al archivo de topología y el segundo a la trayectoria. Además, debemos especificar el número de frames de la simulación como el tamaño de la trayectoria.

> Si no estás familiarizado con los conceptos o la sintaxis de este código, ¡no te preocupes! 💪	Esta es una introducción y los aspectos específicos de este paquete se revisarán en tutoriales posteriores 🎈.


In [ ]:
# Instance of the Universe class
u = mda.Universe(top, traj_end)

# The number of frames in the simulation
number_frames_analysis = len(u.trajectory)

if number_frames_analysis > 10:
  stride_animation = number_frames_analysis/100
else:
  stride_animation = 1

Estas clases serán necesarias para leer y crear los archivos PDB de la simulación.


In [ ]:
# Helper classes to read and get PDB fields
import warnings
warnings.filterwarnings('ignore')
!rm [0-9]?.pdb 2> /dev/null

class Atom(dict):
  def __init__(self, line):
    self["type"] = line[0:6].strip()
    self["idx"] = line[6:11].strip()
    self["name"] = line[12:16].strip()
    self["resname"] = line[17:20].strip()
    self["resid"] = int(int(line[22:26]))
    self["x"] = float(line[30:38])
    self["y"] = float(line[38:46])
    self["z"] = float(line[46:54])
    self["sym"] = line[76:78].strip()

  def __str__(self):
    line = list(" " * 80)
    line[0:6] = self["type"].ljust(6)
    line[6:11] = self["idx"].ljust(5)
    line[12:16] = self["name"].ljust(4)
    line[17:20] = self["resname"].ljust(3)
    line[22:26] = str(self["resid"]).ljust(4)
    line[30:38] = str(self["x"]).rjust(8)
    line[38:46] = str(self["y"]).rjust(8)
    line[46:54] = str(self["z"]).rjust(8)
    line[76:78] = self["sym"].rjust(2)
    return "".join(line) + "\n"

class Molecule(list):
  def __init__(self, file):
    for line in file:
      if "ATOM" in line or "HETATM" in line:
        self.append(Atom(line))

    def __str__(self):
      outstr = ""
      for at in self:
        outstr += str(at)
      return outstr

En el siguiente paso, dividiremos las posiciones de la trayectoria en fotogramas para la animación. Para cada fotograma, guardaremos un archivo PDB (aparecerán muchos archivos en tu directorio de trabajo, no te preocupes).


In [ ]:
# Write out frames for animation
# IMPORTANT: This line will filter the WATER molecules.
protein = u.select_atoms('not (resname WAT)')
i = 0
for ts in u.trajectory[0:len(u.trajectory):int(stride_animation)]:
    if i > -1:
        with mda.Writer('' + str(i) + '.pdb', protein.n_atoms) as W:
            W.write(protein)
    i = i + 1
# Load frames as molecules (py3Dmol let us visualize a single "molecule" per frame)
molecules = []
for i in range(int(len(u.trajectory)/int(stride_animation))):
    with open('' + str(i) + '.pdb') as ifile:
        molecules.append(Molecule(ifile))

models = ""
for i in range(len(molecules)):
  models += "MODEL " + str(i) + "\n"
  for j,mol in enumerate(molecules[i]):
    models += str(mol)
  models += "ENDMDL\n"

Finalmente, crearemos nuestra animación de la trayectoria. Esta sección tomará algún tiempo (**~1 minuto**).


In [ ]:
# Animation
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(models)
for i, at in enumerate(molecules[0]):
    default = {"cartoon": {'color': 'spectrum'}}
    view.setStyle({'model': -1, 'serial': i+1}, at.get("pymol", default))

view.zoomTo()
# We can make an infinite loop with the animation or reduce the animation time with the reps argument.
# reps: 0 means infinite loop
view.animate({'loop': "forward", 'reps': 2})
view.show()

Podemos encapsular todos los bloques de código en una sola función para facilitar su uso.


In [ ]:
def MD_visualization(top, traj_end):
  """
  Inputs:
  top : path to the topology file
  traj_end : path to the trajectory file
  """
  # Instance of the Universe class
  u = mda.Universe(top, traj_end)

  # The number of frames in the simulation
  number_frames_analysis = len(u.trajectory)
  if number_frames_analysis > 10:
    stride_animation = number_frames_analysis/100
  else:
    stride_animation = 1

  # Deleting previously stored frames as PDBs and removing warnings
  import warnings
  warnings.filterwarnings('ignore')
  !rm [0-9]?.pdb 2> /dev/null

    # Helper classes to read and get PDB fields
  class Atom(dict):
    def __init__(self, line):
      self["type"] = line[0:6].strip()
      self["idx"] = line[6:11].strip()
      self["name"] = line[12:16].strip()
      self["resname"] = line[17:20].strip()
      self["resid"] = int(int(line[22:26]))
      self["x"] = float(line[30:38])
      self["y"] = float(line[38:46])
      self["z"] = float(line[46:54])
      self["sym"] = line[76:78].strip()

    def __str__(self):
      line = list(" " * 80)
      line[0:6] = self["type"].ljust(6)
      line[6:11] = self["idx"].ljust(5)
      line[12:16] = self["name"].ljust(4)
      line[17:20] = self["resname"].ljust(3)
      line[22:26] = str(self["resid"]).ljust(4)
      line[30:38] = str(self["x"]).rjust(8)
      line[38:46] = str(self["y"]).rjust(8)
      line[46:54] = str(self["z"]).rjust(8)
      line[76:78] = self["sym"].rjust(2)
      return "".join(line) + "\n"

  class Molecule(list):
    def __init__(self, file):
      for line in file:
        if "ATOM" in line or "HETATM" in line:
          self.append(Atom(line))

      def __str__(self):
        outstr = ""
        for at in self:
          outstr += str(at)
        return outstr
  # Write out frames for animation
  # IMPORTANT: This line will filter the WATER molecules.
  protein = u.select_atoms('not (resname WAT)')
  i = 0
  for ts in u.trajectory[0:len(u.trajectory):int(stride_animation)]:
      if i > -1:
          with mda.Writer('' + str(i) + '.pdb', protein.n_atoms) as W:
              W.write(protein)
      i = i + 1
  # Load frames as molecules (py3Dmol let us visualize a single "molecule" per frame)
  molecules = []
  for i in range(int(len(u.trajectory)/int(stride_animation))):
      with open('' + str(i) + '.pdb') as ifile:
          molecules.append(Molecule(ifile))

  models = ""
  for i in range(len(molecules)):
    models += "MODEL " + str(i) + "\n"
    for j,mol in enumerate(molecules[i]):
      models += str(mol)
    models += "ENDMDL\n"

  # Animation
  view = py3Dmol.view(width=800, height=600)
  view.addModelsAsFrames(models)
  for i, at in enumerate(molecules[0]):
      default = {"cartoon": {'color': 'spectrum'}}
      view.setStyle({'model': -1, 'serial': i+1}, at.get("pymol", default))

  view.zoomTo()
  view.animate({'loop': "forward", 'reps': 2})
  return view

In [ ]:
view = MD_visualization(top, traj_end)
view.show()

## 2. Usando la versión web de NGLview

Utilizaremos una versión web de **NGLview** para este propósito. NGLview es un visualizador interactivo de moléculas y trayectorias. Para ver cómo funciona NGLview, carga los archivos descargados del repositorio y luego explora las selecciones y representaciones dentro de la versión web de [NGL viewer](http://nglviewer.org/ngl/)


## 3. [EXPERIMENTAL] Using the *nglview* python library




**NGLview** tiene un widget para Python y Jupyter Notebooks. Desafortunadamente, la visualización de trayectorias no funciona en Google Colab. Con suerte, esto se solucionará pronto, así que te dejamos un ejemplo de su uso.


In [ ]:
!pip install nglview
import nglview as nv

# This is esential in order to display de nglview widget in Colab.
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
protein = mda.Universe(top, traj_end)
view = nv.show_mdanalysis(protein)
view

## Apéndice B. *Crear una representación al estilo Goodsell en la visualización tridimensional de proteínas*


Puedes obtener hermosas representaciones al estilo de PDB 101 en unos pocos clics utilizando la aplicación web [3D Protein Imaging](https://3dproteinimaging.com/protein-imager).

Intenta descargar una de las cadenas PDB de la sesión de Google Colab y súbela a 3D Protein Imaging. Luego, puedes enviarnos tu ilustración y podemos darte retroalimentación.
<figure><center><img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/pdbformat_05.png'></figure></center>


# Apéndice C. Ejemplos de py3Dmol

Nota: Este apéndice corresponde al tutorial completo de py3Dmol y está disponible [**aquí**](https://pypi.org/project/py3Dmol/). Dado que ha sido creado para funcionar en IPython/Jupyter, existe la posibilidad de que algunas secciones no funcionen en Google Colab.



Un sencillo widget [IPython/Jupyter](http://jupyter.org/) para incrustar un visor interactivo [3Dmol.js](http://3dmol.csb.pitt.edu) en un cuaderno.

El widget es completamente estático, lo que significa que el visor no necesita un kernel de IPython en ejecución para ser útil y las páginas web y presentaciones generadas desde el cuaderno funcionarán según lo esperado. Sin embargo, esto también significa que solo hay comunicación unidireccional entre el cuaderno y el visor.

## Instalación
------------

De PyPI:

    !pip install py3Dmol
    
API
---

El objeto de vista devuelto tiene la misma API que [3Dmol.GLViewer](http://3dmol.csb.pitt.edu/doc/$3Dmol.GLViewer.html) con la excepción de que las funciones devuelven None.


In [ ]:
## Aquí escribe el código para instalar py3Dmol
!pip install py3Dmol

In [ ]:
import py3Dmol

In [ ]:
p = py3Dmol.view(query='mmtf:1ycr')
p.setStyle({'cartoon': {'color':'spectrum'}})
p.addStyle({'within':{'distance': 5, 'sel':{'resi':21}}},{'stick':{'colorscheme':'blue'}})


In [ ]:
xyz = '''4
* (null), Energy   -1000.0000000
N     0.000005    0.019779   -0.000003   -0.157114    0.000052   -0.012746
H     0.931955   -0.364989    0.000003    1.507100   -0.601158   -0.004108
H    -0.465975   -0.364992    0.807088    0.283368    0.257996   -0.583024
H    -0.465979   -0.364991   -0.807088    0.392764    0.342436    0.764260
'''

In [ ]:
xyzview = py3Dmol.view(width=400,height=400)
xyzview.addModel(xyz,'xyz',{'vibrate': {'frames':10,'amplitude':1}})
xyzview.setStyle({'stick':{}})
xyzview.setBackgroundColor('0xeeeeee')
xyzview.animate({'loop': 'backAndForth'})
xyzview.zoomTo()
xyzview.show()

Mostrar archivo local.

In [ ]:
benz='''
     RDKit          3D

  6  6  0  0  0  0  0  0  0  0999 V2000
   -0.9517    0.7811   -0.6622 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.2847    1.3329   -0.3121 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.2365    0.5518    0.3512 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.9517   -0.7811    0.6644 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.2847   -1.3329    0.3144 C   0  0  0  0  0  0  0  0  0  0  0  0
   -1.2365   -0.5518   -0.3489 C   0  0  0  0  0  0  0  0  0  0  0  0
  1  2  2  0
  2  3  1  0
  3  4  2  0
  4  5  1  0
  5  6  2  0
  6  1  1  0
M  END
$$$$'''
view = py3Dmol.view()
view.addModel(benz,'sdf')
view.setStyle({'stick':{}})
view.setStyle({'model':0},{'stick':{'colorscheme':'cyanCarbon'}})
view.zoomTo()

Puedes crear un único objeto de lienzo con varios visores dispuestos en una cuadrícula (3Dmol.createViewerGrid).


In [ ]:
view = py3Dmol.view(query='pdb:1dc9',linked=False,viewergrid=(2,2))
view.setViewStyle({'style':'outline','color':'black','width':0.1})
view.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}},viewer=(0,1))
view.setStyle({'stick':{'colorscheme':'greenCarbon'}},viewer=(1,0))
view.setStyle({'cartoon':{'color':'spectrum'}},viewer=(1,1))
view.removeAllModels(viewer=(0,0))
view.addModel(benz,'sdf',viewer=(0,0))
view.setStyle({'stick':{}},viewer=(0,0))
view.zoomTo(viewer=(0,0))
view.render()

In [ ]:
view = py3Dmol.view(query='pdb:1ycr')
chA = {'chain':'A'}
chB = {'chain':'B'}
view.setStyle(chA,{'cartoon': {'color':'spectrum'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.7,'color':'white'}, chA)
view.setStyle(chB,{'stick':{}})
view.show()

In [ ]:
view = py3Dmol.view(query='pdb:5ire',options={'doAssembly':True})
view.setStyle({'cartoon':{'color':'spectrum'}})
view.show()

Colorea por factores de temperatura

In [ ]:
view = py3Dmol.view(query='pdb:1ycr')
view.setStyle({'cartoon': {'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.7,'colorscheme':{'prop':'b','gradient':'sinebow','min':0,'max':70}})

Generar una imagen en línea de lo que se encuentra actualmente en el visor (todo blanco si la estructura aún no se ha cargado).


In [ ]:
png = view.png()
png

In [ ]:
import requests, base64
r = requests.get('https://mmtf.rcsb.org/v1.0/full/5lgo')
view = py3Dmol.view()
view.addModel(base64.b64encode(r.content).decode(),'mmtf')
view.addUnitCell()
view.zoomTo()

In [ ]:
!pip install py3Dmol


In [ ]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.3-Linux-x86_64.sh
!time bash ./Miniconda3-py37_4.8.3-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
import py3Dmol

def MolTo3DView(mol, size=(300, 300), style="stick", surface=False, opacity=0.5):
    """Draw molecule in 3D

    Args:
    ----
        mol: rdMol, molecule to show
        size: tuple(int, int), canvas size
        style: str, type of drawing molecule
               style can be 'line', 'stick', 'sphere', 'carton'
        surface, bool, display SAS
        opacity, float, opacity of surface, range 0.0-1.0
    Return:
    ----
        viewer: py3Dmol.view, a class for constructing embedded 3Dmol.js views in ipython notebooks.
    """
    assert style in ('line', 'stick', 'sphere', 'carton')
    mblock = Chem.MolToMolBlock(mol)
    viewer = py3Dmol.view(width=size[0], height=size[1])
    viewer.addModel(mblock, 'mol')
    viewer.setStyle({style:{}})
    if surface:
        viewer.addSurface(py3Dmol.SAS, {'opacity': opacity})
    viewer.zoomTo()
    return viewer

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem

def smi2conf(smiles):
    '''Convert SMILES to rdkit.Mol with 3D coordinates'''
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol)
        AllChem.MMFFOptimizeMolecule(mol, maxIters=200)
        return mol
    else:
        return None

smi = 'COc3nc(OCc2ccc(C#N)c(c1ccc(C(=O)O)cc1)c2P(=O)(O)O)ccc3C[NH2+]CC(I)NC(=O)C(F)(Cl)Br'
conf = smi2conf(smi)
viewer = MolTo3DView(conf, size=(600, 300), style='sphere')
viewer.show()

In [ ]:
viewer = MolTo3DView(conf, size=(600, 300), style='sphere')
viewer.show()

In [ ]:
#!pip install ipywidgets
%reload_ext autoreload
%autoreload 2
from ipywidgets import interact,fixed,IntSlider
import ipywidgets

smis = [ 'COc3nc(OCc2ccc(C#N)c(c1ccc(C(=O)O)cc1)c2P(=O)(O)O)ccc3C[NH2+]CC(I)NC(=O)C(F)(Cl)Br',
    'CC(NCCNCC1=CC=C(OCC2=C(C)C(C3=CC=CC=C3)=CC=C2)N=C1OC)=O',
    'Cc1c(COc2cc(OCc3cccc(c3)C#N)c(CN3C[C@H](O)C[C@H]3C(O)=O)cc2Cl)cccc1-c1ccc2OCCOc2c1',
    'CCCCC(=O)NCCCCC(=O)NCCCCCC(=O)[O-]',
    "CC(NCCNCC1=CC=C(OCC2=C(C)C(C3=CC=CC=C3)=CC=C2)N=C1OC)=O"]

confs = [smi2conf(s) for s in smis]

def conf_viewer(idx):
    mol = confs[idx]
    return MolTo3DView(mol).show()

interact(conf_viewer, idx=ipywidgets.IntSlider(min=0,max=4-1, step=1))

In [ ]:
%reload_ext autoreload
%autoreload 2

from ipywidgets import interact,fixed,IntSlider,interact_manual
import ipywidgets

smis = [ 'COc3nc(OCc2ccc(C#N)c(c1ccc(C(=O)O)cc1)c2P(=O)(O)O)ccc3C[NH2+]CC(I)NC(=O)C(F)(Cl)Br',
    'CC(NCCNCC1=CC=C(OCC2=C(C)C(C3=CC=CC=C3)=CC=C2)N=C1OC)=O',
    'Cc1c(COc2cc(OCc3cccc(c3)C#N)c(CN3C[C@H](O)C[C@H]3C(O)=O)cc2Cl)cccc1-c1ccc2OCCOc2c1',
    'CCCCC(=O)NCCCCC(=O)NCCCCCC(=O)[O-]',
    "CC(NCCNCC1=CC=C(OCC2=C(C)C(C3=CC=CC=C3)=CC=C2)N=C1OC)=O"]

confs = [smi2conf(s) for s in smis]

def style_selector(idx, s):
    conf = confs[idx]
    return MolTo3DView(conf, style=s).show()

